In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import copy
import random
from sklearn.metrics import f1_score

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd drive/Shared\ drives/ISE540/Data/Problem2

/content/drive/Shared drives/ISE540/Data/Problem2


In [ ]:
df = pd.read_csv("Web_content_cleaned.csv")

In [ ]:
df.head()

,Unnamed: 0,ID,Text,Domain,URL,Content
0,0,1.324130e+18,“For Premier Doug Ford to say that they've don...,Medical,https://t.co/vk89LoQfZp,And it's not just Ontario. Long lines and shor...
1,1,1.324130e+18,"In any season, getting sick is tough. But rig...",Medical,https://t.co/OsHSfvef52,AboutPressCopyrightContact usCreatorsAdvertise...
2,2,1.324130e+18,“For Premier Doug Ford to say that they've don...,Medical,https://t.co/vk89LoQfZp,And it's not just Ontario. Long lines and shor...
3,3,1.324130e+18,@FatEmperor And the lastest vicim of the swine...,Medical,https://t.co/HPkErToAHc,"To date, more than 80 cases have been lodged b..."
4,4,1.324130e+18,Fargo Cass Public Health holds additional flu ...,Medical,https://t.co/ztEuaglz07,FARGO — Due to the positive response to last w...


In [ ]:
df_unique_text = df.drop_duplicates(subset = ["Text"])

In [ ]:
df_unique_text = df_unique_text.drop(columns=["Unnamed: 0"])

In [ ]:
df_unique_text.reset_index(drop=True, inplace=True)

In [ ]:
df_sampling = df_unique_text

In [ ]:
med_index = list(df_sampling[df_sampling.Domain == "Medical"].index)
game_index = list(df_sampling[df_sampling.Domain == "Video_Games"].index)
sport_index = list(df_sampling[df_sampling.Domain == "Sports"].index)
unknown_index = list(df_sampling[df_sampling.Domain == "unknown"].index)

In [ ]:
def random_sample(domain):
  if(domain == "Medical"):
    return med_index
  elif(domain == "Video_Games"):
    return game_index
  elif(domain == "Sports"):
    return sport_index
  elif(domain == "unknown"):
    return unknown_index

In [ ]:
for i in range(0,df_sampling.shape[0]):
  id = df_sampling["ID"][i]
  text = df_sampling["Text"][i]
  domain = df_sampling["Domain"][i]
  temp = copy.deepcopy(random_sample(domain))
  temp.remove(i)
  for j in range(0,9):
    random_i = random.choice(temp)
    URL = df_sampling["URL"][random_i]
    Content = df_sampling["Content"][random_i]
    df_sampling = df_sampling.append({"ID": id, "Text": text, "Domain": domain, "URL": URL, "Content": Content}, ignore_index=True)

In [ ]:
df_sampling.to_csv("Sampled_unbalanced.csv", encoding = "utf-8-sig")

In [ ]:
df_sampling["Match"] = 0
for i in range(0,1318):
  df_sampling["Match"][i] = 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
df_sampling["Match"].value_counts()

0    11862
1     1318
Name: Match, dtype: int64

In [ ]:
df_unique_text["Match"] = 1

In [ ]:
df_sampling_balanced = df_sampling

In [ ]:
for i in range(0,8):
  df_sampling_balanced = df_sampling_balanced.append(df_unique_text)

In [ ]:
df_sampling_balanced = df_sampling_balanced.reset_index()

In [ ]:
df_sampling_balanced.to_csv("Sampled_balanced.csv", encoding = "utf-8-sig")

Vectorizing the tweets and contents

In [ ]:
vectorizer_tweet = TfidfVectorizer(max_df = 200,max_features=6000)
vectorizer_content = TfidfVectorizer(max_df = 200,max_features=6000)

In [ ]:
tweet_tfidf = vectorizer_tweet.fit_transform(df_sampling["Text"])

In [ ]:
content_tfidf = vectorizer_content.fit_transform(df_sampling["Content"])

In [ ]:
import scipy.sparse as sp

In [ ]:
A_tweet_tfidf = tweet_tfidf.A
A_content_tfidf = content_tfidf.A

In [ ]:
A_content_tfidf.shape

(13180, 6000)

In [ ]:
concat_tfidf =np.hstack((A_tweet_tfidf, A_content_tfidf))

In [ ]:
y = df_sampling["Match"]

In [ ]:
len(y)

13180

In [ ]:
X_train, X_test, y_train, y_test =  train_test_split(concat_tfidf, y, test_size=0.3, random_state=42,stratify = y)

Log Regression

In [ ]:
# Balanced

In [ ]:
clf1 = LogisticRegression(class_weight='balanced').fit(X_train, y_train)

In [ ]:
y_pred1 =clf1.predict(X_test)
clf1.score(X_test,y_test)

0.4772382397572079

In [ ]:
f1_score(y_test, y_pred1)

0.0009666505558240696

In [ ]:
# Unbalanced

In [ ]:
clf2 = LogisticRegression().fit(X_train, y_train)

In [ ]:
y_pred2 = clf2.predict(X_test)
clf2.score(X_test,y_test)

0.9001011633788568

Log Regression with Balanced samplling

In [ ]:
vectorizer_tweet_bal = TfidfVectorizer(max_df = 200,max_features=6000)
vectorizer_content_bal = TfidfVectorizer(max_df = 200,max_features=6000)

In [ ]:
tweet_tfidf_bal = vectorizer_tweet_bal.fit_transform(df_sampling_balanced["Text"])
content_tfidf_bal = vectorizer_content_bal.fit_transform(df_sampling_balanced["Content"])

In [ ]:
A_tweet_tfidf_bal = tweet_tfidf_bal.A
A_content_tfidf_bal = content_tfidf_bal.A
concat_tfidf_bal =np.hstack((A_tweet_tfidf_bal, A_content_tfidf_bal))

In [ ]:
y_bal = df_sampling_balanced["Match"]

In [ ]:
X_train_bal, X_test_bal, y_train_bal, y_test_bal =  train_test_split(concat_tfidf_bal, y_bal, test_size=0.3, random_state=42,stratify = y_bal)

In [ ]:
concat_tfidf_bal.shape

(23724, 12000)

In [ ]:
# Balanced

In [ ]:
clf1 = LogisticRegression(class_weight='balanced').fit(X_train_bal, y_train_bal)

In [ ]:
y_pred1 =clf1.predict(X_test_bal)
clf1.score(X_test_bal,y_test_bal)

0.40573194717617306

In [ ]:
f1_score(y_test_bal, y_pred1)

0.4167126309983453

MLP

In [ ]:
from sklearn.neural_network import MLPClassifier
model3 = MLPClassifier()
model3.fit(X_train_bal, y_train_bal)

In [ ]:
y_pred=model3.predict(X_test_bal)
y_train_pred=model3.predict(X_train_bal)
model3.score(X_test_bal,y_test_bal)

0.9682495082888452

In [ ]:
f1_score(y_test_bal, y_pred)

0.9691678035470669

Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeClassifier
model4 = DecisionTreeClassifier(class_weight='balanced')
model4.fit(X_train_bal, y_train_bal)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight='balanced', criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
y_pred=model4.predict(X_test_bal)
model4.score(X_test_bal,y_test_bal)

0.9096656364147232

In [ ]:
f1_score(y_test_bal, y_pred)

0.9170001290822254